In [1]:
import openmm as mm
import numpy as np
import openmm.app as app
import simtk.unit as unit
from openmmforcefields.generators import GAFFTemplateGenerator as gen
from openff.toolkit import Topology
import pytraj as pt
import os
import matplotlib.pyplot as plt

from openmm.app import CharmmPsfFile, CharmmCrdFile, CharmmParameterSet



In [2]:
base = '/scratch/htc/fsafarov/structures/7rqz_sep_2025/7qrz'

#We use psf and crd system because pdb files does not support the 6-digit base system for ATOM ID's, which is in our case utilized for POPC. 
#Input below is from psf and crd from CHARMM-GUI.
exit
psf = CharmmPsfFile(os.path.join(base, 'step5_assembly.psf'))
crd = CharmmCrdFile(os.path.join(base, 'step5_assembly.crd'))

#Input all the parameters from toppar folder of CHARMM-GUI output for all the components of psf and crd.

params = CharmmParameterSet(
    os.path.join(base, 'toppar/top_all36_prot.rtf'),
    os.path.join(base, 'toppar/par_all36m_prot.prm'),
    os.path.join(base, 'toppar/top_all36_na.rtf'),
    os.path.join(base, 'toppar/par_all36_na.prm'),
    os.path.join(base, 'toppar/top_all36_carb.rtf'),
    os.path.join(base, 'toppar/par_all36_carb.prm'),
    os.path.join(base, 'toppar/top_all36_lipid.rtf'),
    os.path.join(base, 'toppar/par_all36_lipid.prm'),
    os.path.join(base, 'toppar/top_all36_cgenff.rtf'),
    os.path.join(base, 'toppar/par_all36_cgenff.prm'),
    os.path.join(base, 'toppar/toppar_all36_moreions.str'),
    os.path.join(base, 'toppar/top_interface.rtf'),
    os.path.join(base, 'toppar/par_interface.prm'),
    os.path.join(base, 'toppar/toppar_all36_nano_lig.str'),
    os.path.join(base, 'toppar/toppar_all36_nano_lig_patch.str'),
    os.path.join(base, 'toppar/toppar_all36_synthetic_polymer.str'),
    os.path.join(base, 'toppar/toppar_all36_synthetic_polymer_patch.str'),
    os.path.join(base, 'toppar/toppar_all36_polymer_solvent.str'),
    os.path.join(base, 'toppar/toppar_water_ions.str'),
    os.path.join(base, 'toppar/toppar_dum_noble_gases.str'),
    os.path.join(base, 'toppar/toppar_ions_won.str'),
    os.path.join(base, 'toppar/cam.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_arg0.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_c36m_d_aminoacids.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_fluoro_alkanes.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_heme.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_na_combined.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_retinol.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_model.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_modify_res.str'),
    os.path.join(base, 'toppar/toppar_all36_na_nad_ppi.str'),
    os.path.join(base, 'toppar/toppar_all36_na_rna_modified.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_sphingo.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_archaeal.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_bacterial.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_cardiolipin.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_cholesterol.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_dag.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_inositol.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_lnp.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_lps.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_mycobacterial.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_miscellaneous.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_model.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_prot.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_tag.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_yeast.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_hmmm.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_detergent.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_ether.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_oxidized.str'),
    os.path.join(base, 'toppar/toppar_all36_carb_glycolipid.str'),
    os.path.join(base, 'toppar/toppar_all36_carb_glycopeptide.str'),
    os.path.join(base, 'toppar/toppar_all36_carb_imlab.str'),
    os.path.join(base, 'toppar/toppar_all36_label_spin.str'),
    os.path.join(base, 'toppar/toppar_all36_label_fluorophore.str'),
    os.path.join(base, '6eu/6eu.rtf'),
    os.path.join(base, '6eu/6eu.prm')
)

In [3]:
state = mm.XmlSerializer.deserialize(open('/scratch/htc/fsafarov/states/trpv1/7rqz/minimization_7lpe.xml').read())
centered_positions = state.getPositions()

In [ ]:
a_vec, b_vec, c_vec = state.getPeriodicBoxVectors()
a_norm, b_norm, c_norm = np.linalg.norm(a_vec), np.linalg.norm(b_vec), np.linalg.norm(c_vec)
psf.setBox(a_norm, b_norm, c_norm)

In [4]:
system = psf.createSystem(params, nonbondedMethod=app.LJPME, nonbondedCutoff=1.0 * unit.nanometer, constraints = app.HBonds)

In [5]:
n1_atom = psf.topology.getNumAtoms()
n1_atom

118294

In [ ]:
protein_restraint = mm.CustomExternalForce('k_2*periodicdistance(x, y, z, x0, y0, z0)^2')

protein_restraint.addGlobalParameter('k_2', 5*unit.kilocalories_per_mole/unit.angstrom**2)
protein_restraint.addPerParticleParameter('x0')
protein_restraint.addPerParticleParameter('y0')
protein_restraint.addPerParticleParameter('z0')
system.addForce(protein_restraint)

In [ ]:
#standard aminoacid residues in proteins (to apply the specific restraint to the protein)
std_amino_acids = ['GLY', 'TYR', 'PHE', 'ARG', 'HIS', 'ALA', 'PRO', 'GLU', 'SER', 'LYS',
    'THR', 'MET', 'CYS', 'LEU', 'GLN', 'ASN', 'VAL', 'ILE', 'ASP', 'TRP']

for residue in psf.topology.residues():
    if residue.name in std_amino_acids:
        for atom in residue.atoms():
            if atom.element.symbol != 'H':
                pos = centered_positions[atom.index] #change made here from centered positions
                protein_restraint.addParticle(atom.index, [pos[0], pos[1], pos[2]])



In [ ]:
restraint = mm.CustomExternalForce('k_1*periodicdistance(x, y, z, x0, y0, z0)^2')

restraint.addGlobalParameter('k_1', 5*unit.kilocalories_per_mole/unit.angstrom**2)
restraint.addPerParticleParameter('x0')
restraint.addPerParticleParameter('y0')
restraint.addPerParticleParameter('z0')
system.addForce(restraint)

In [ ]:
for residue in psf.topology.residues():
    if residue.name == 'POPC':
        for atom in residue.atoms():
            if atom.element.symbol != 'H':
                pos = centered_positions[atom.index] #change made here from centered positions
                restraint.addParticle(atom.index, [pos[0], pos[1], pos[2]])

In [ ]:
mem_potential = mm.CustomExternalForce("q*Ez*(z - z0)")
mem_potential.addGlobalParameter('Ez', 0.05 * unit.volt / unit.nanometer)
mem_potential.addPerParticleParameter('q')
mem_potential.addPerParticleParameter('z0')
system.addForce(mem_potential)

In [ ]:
def find_nonbonded(system):
    for f in system.getForces():
        if isinstance(f, mm.NonbondedForce):
            return f

nb_force = find_nonbonded(system)

In [ ]:
for residue in psf.topology.residues():
    if residue.name in ['CAL', 'SOD', 'CLA']:
        for atom in residue.atoms():
            q,_,_ = nb_force.getParticleParameters(atom.index)
            mem_potential.addParticle(atom.index, [q, pos[2]])

In [6]:
integrator_re = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 1.0*unit.femtoseconds)

In [7]:
platform = mm.Platform.getPlatformByName('CUDA')
# #to run the simulation on several GPU's parallelly
properties  = {'CudaDeviceIndex': '0,1,2', 'CudaPrecision': 'mixed'}

In [8]:
simulation_re = app.Simulation(psf.topology, system, integrator_re, platform, properties)
simulation_re.loadState('/scratch/htc/fsafarov/states/trpv1/7rqz/minimization_7lpe.xml')


In [9]:
system.addForce(mm.MonteCarloBarostat(1.0*unit.atmosphere, 300*unit.kelvin, 100))

11

In [10]:
state_1 = simulation_re.context.getState(getVelocities=True)
velocities = state_1.getVelocities(asNumpy = True)

In [11]:
dict(simulation_re.context.getParameters()) #check the parameters!!!

{'k_1': 0.0, 'k_2': 0.0}

In [12]:
simulation_re.context.setParameter('k_2', 1.0*unit.kilocalories_per_mole/unit.angstrom**2)
simulation_re.context.setParameter('k_1', 1.0*unit.kilocalories_per_mole/unit.angstrom**2)

In [13]:
simulation_re.context.reinitialize(preserveState=True)
simulation_re.context.setVelocities(velocities)
simulation_re.context.setPositions(centered_positions)

In [14]:
print(dict(simulation_re.context.getParameters())) #check the parameters!!!

{'MonteCarloPressure': 1.01325,
 'MonteCarloTemperature': 300.0,
 'k_1': 418.40000000000003,
 'k_2': 418.40000000000003}

In [15]:
state_1 = simulation_re.context.getState(getForces=True)
forces = state_1.getForces(asNumpy = True).value_in_unit(unit.kilojoule/unit.nanometer/unit.mole)
norm = np.linalg.norm(forces, axis = 1)
max_force = max(norm)
np.argmax(norm), np.max(norm)

(50287, 2498.3152781685835)

In [16]:
simulation_re.reporters.append(app.DCDReporter('/scratch/htc/fsafarov/traj/traj_trp/mor_amber_protocol_2/7rqz/step6_relax.dcd', 1000, enforcePeriodicBox=True))
simulation_re.reporters.append(app.CheckpointReporter('/scratch/htc/fsafarov/traj/traj_trp/mor_amber_protocol_2/7rqz/step6.chk', 1000))
simulation_re.step(5000)


In [17]:
simulation_re.saveState('/scratch/htc/fsafarov/states/trpv1/7rqz/relaxation.xml')

In [18]:
with open('/scratch/htc/fsafarov/states/trpv1/7rqz/relaxed_system.xml', 'w') as f:
    f.write(mm.XmlSerializer.serialize(system))